In [1]:
import sys
import pprint
import numpy as np
import pandas as pd
import datetime
import requests
import json
from pandas.io.json import json_normalize
import os
import pymongo

In [1]:
client = pymongo.MongoClient()
client = pymongo.MongoClient("mongodb://localhost:27017/")
database = client.aggregated
btc = database.BTC_historical

NameError: name 'pymongo' is not defined

In [38]:
database.list_collection_names()

['storage', 'BTC_historical', 'BTC']

In [44]:
cursor = database.BTC.find({})

In [54]:
len(list(database.BTC.find()))

1490

In [55]:
df = pd.DataFrame(list(database.BTC.find()))

In [63]:
df2 = json_normalize(list(database.BTC.find()))

In [96]:
database.BTC.find_one()

{'_id': ObjectId('5b46fcca4ca32000090d962c'),
 'date': datetime.datetime(2018, 7, 12, 7, 1, 0, 19000),
 'ETH': {'p': 0.0698, 'v': 479464.30963131477},
 'TRX': {'p': 5.21e-06, 'v': 2130306290.4260561},
 'XRP': {'p': 7.017e-05, 'v': 115938017.73223434},
 'NCASH': {'p': 1.88e-06, 'v': 70176086},
 'NEO': {'p': 0.005142, 'v': 761382.6702750317},
 'BNB': {'p': 0.001977, 'v': 693064.2500000003},
 'ADA': {'p': 2.02e-05, 'v': 116197948.82639325},
 'NANO': {'p': 0.0003695, 'v': 1470930.4617778405},
 'ICX': {'p': 0.0002214, 'v': 11908161.875029901},
 'QTUM': {'p': 0.001206, 'v': 3000588.3890464026},
 'LTC': {'p': 0.01207, 'v': 1116656.5271089445},
 'XLM': {'p': 2.951e-05, 'v': 76266938.54099071},
 'VIBE': {'p': 1.092e-05, 'v': 1634266},
 'IOTA': {'p': 0.00015921, 'v': 0},
 'BCC': {'p': 0.09311111, 'v': 0},
 'DGD': {'p': 0.01468, 'v': 6005.119715927146},
 'VEN': {'p': 0.0003414, 'v': 19316296.158355776},
 'GVT': {'p': 0.001159, 'v': 248425.13318400003},
 'AION': {'p': 0.0001436, 'v': 1221760.97270

In [94]:
for element in database.BTC.find({"date" : datetime.datetime(2018, 7, 12, 7, 1, 0, 19000)}):
    print(element)

{'_id': ObjectId('5b46fcca4ca32000090d962c'), 'date': datetime.datetime(2018, 7, 12, 7, 1, 0, 19000), 'ETH': {'p': 0.0698, 'v': 479464.30963131477}, 'TRX': {'p': 5.21e-06, 'v': 2130306290.4260561}, 'XRP': {'p': 7.017e-05, 'v': 115938017.73223434}, 'NCASH': {'p': 1.88e-06, 'v': 70176086}, 'NEO': {'p': 0.005142, 'v': 761382.6702750317}, 'BNB': {'p': 0.001977, 'v': 693064.2500000003}, 'ADA': {'p': 2.02e-05, 'v': 116197948.82639325}, 'NANO': {'p': 0.0003695, 'v': 1470930.4617778405}, 'ICX': {'p': 0.0002214, 'v': 11908161.875029901}, 'QTUM': {'p': 0.001206, 'v': 3000588.3890464026}, 'LTC': {'p': 0.01207, 'v': 1116656.5271089445}, 'XLM': {'p': 2.951e-05, 'v': 76266938.54099071}, 'VIBE': {'p': 1.092e-05, 'v': 1634266}, 'IOTA': {'p': 0.00015921, 'v': 0}, 'BCC': {'p': 0.09311111, 'v': 0}, 'DGD': {'p': 0.01468, 'v': 6005.119715927146}, 'VEN': {'p': 0.0003414, 'v': 19316296.158355776}, 'GVT': {'p': 0.001159, 'v': 248425.13318400003}, 'AION': {'p': 0.0001436, 'v': 1221760.9727028697}, 'BCPT': {'p'

In [93]:
database.BTC.find().sort({_id:1}).limit(1)

NameError: name '_id' is not defined

In [153]:
def data_load_interval(seq_len, gap):
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client.aggregated
    collection = db.BTC_historical

    #  now = datetime.datetime.utcnow()   # to be used only in production
    
    now = datetime.datetime(2018, 7, 12, 7, 1, 0) # needs to be removed in production
    
    data = [collection.find_one({"date" : {'$gte': now-datetime.timedelta(minutes = 1)}})]
    gap = 60
    margin = 10   # in case there are missings
    
    for i in range(seq_len-1 + margin):
    
        start = now - datetime.timedelta(minutes = gap * (i+1) + 1)
        end = now - datetime.timedelta(minutes = gap * (i+1))
        entry = collection.find_one({"date" : {'$lt': end, '$gte': start}})
        
        if entry is None:
            pass
        
        else:
            data.append(entry)
   
    df = json_normalize(data).transpose()
    df.columns = df.loc["date"]
    df = df.drop(["_id", "date"])
    rdf = df[df.columns[::-1]]
    
    return rdf.iloc[:,-seq_len:]

In [154]:
start = datetime.datetime.now()
df = data_load_interval(100, 60)
print(datetime.datetime.now()-start)

0:00:11.843386


In [11]:
import pymongo
from pandas.io.json import json_normalize

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client.aggregated
collection = db.BTC_historical

In [3]:
data = collection.find({})

In [5]:
type(data[:50000])

pymongo.cursor.Cursor

In [176]:
nd = []
for element in data:
    nd.append(element)

In [3]:
half = datetime.datetime.now() - datetime.timedelta(days = 90)

In [12]:
df = json_normalize(list(data[:50000])).transpose()

KeyboardInterrupt: 

In [14]:
df.columns = df.loc["date"]
df = df.drop(["_id", "date"])
df.to_csv("new_data_btc_3.csv")

In [186]:
df.iloc[:,:20000].to_csv("new_data_btc_small.csv")

In [18]:
df1.shape

(720, 64696)

In [11]:
data = list(collection.find({"date" : { '$gte': half + datetime.timedelta(days = 45)}}))
df = json_normalize(data)

In [12]:
df = df.transpose()

In [16]:
df1 = pd.read_csv("new_data_btc_2.csv", index_col = 0)

In [20]:
df_tot = df1.join(df)

In [21]:
df_tot.shape

(720, 126017)

In [1]:
import pandas as pd
df = pd.read_csv("new_data_4.csv", index_col = 0)

In [5]:
df.iloc[:,::20].index

Index(['1ST.p', '1ST.v', '2GIVE.p', '2GIVE.v', '8BIT.p', '8BIT.v', 'ABY.p',
       'ABY.v', 'ADA.p', 'ADA.v',
       ...
       'ZCL.p', 'ZCL.v', 'ZEC.p', 'ZEC.v', 'ZEN.p', 'ZEN.v', 'ZIL.p', 'ZIL.v',
       'ZRX.p', 'ZRX.v'],
      dtype='object', length=720)

In [3]:
df.iloc[:,::20].to_csv("data_20min.csv")

In [4]:
df1 = pd.read_csv("new_data_btc_1.csv", index_col = 0)

In [11]:
df_tot.shape

(724, 4000)

In [6]:
df1 = df1.iloc[:,::60]

In [8]:
df2 = pd.read_csv("new_data_4.csv", index_col = 0)

In [9]:
df2 = df2.iloc[:,::60]

In [10]:
df_tot = df1.join(df2)

In [2]:
amountOfHoursToLoadInHourlyStrategy = 100

In [3]:
now = datetime.datetime.utcnow()

In [5]:
df = pd.DataFrame()

In [6]:
findDate = now + datetime.timedelta(hours =- int(amountOfHoursToLoadInHourlyStrategy)+1)

In [7]:
findDate

datetime.datetime(2018, 9, 3, 12, 58, 34, 245382)

In [8]:
untilBackup = findDate + datetime.timedelta(minutes =+ 5)

In [9]:
untilBackup

datetime.datetime(2018, 9, 3, 13, 3, 34, 245382)

In [13]:
datetime.timedelta(minutes = 5)

datetime.timedelta(0, 300)

In [8]:
50000/24/60

34.72222222222222